In [ ]:
import concurrent.futures
import os
from functools import partial

import matplotlib.pyplot as plt
import numpy as np

from materials import get_materials_Cvec
from safe_module import distance
from themes import symmetry_classes

In [ ]:
plt.rcParams['lines.markersize'] = 3

In [ ]:
write = False

In [ ]:
data_size = 10000

In [ ]:
Cvec_Brown, Cvec_Brown_stdv, *_ = get_materials_Cvec(material="Brown")

In [ ]:
# This should ideally come from a grid search

beta0 = np.zeros(7)

for i, sigma in enumerate(symmetry_classes):
    temp = distance(Cvec_Brown, sigma)
    beta0[i] = temp[0]
    print(f'Distance to {sigma} = {beta0[i]:.1f}')

In [ ]:
#beta0 = [26, 21.1, 18.6, 16.2, 7.9, 6.4, 3.8]

In [ ]:
# Writing Cmat realizations to file

if write:
    with open('output.txt', 'w') as f:
        f.write('C11 \tC12 \tC13 \tC14 \tC15 \tC16 \tC22 \tC23 \tC24 \tC25 \tC26 \tC33 \tC34 \tC35 \tC36 \tC44 '
             '\tC45 \tC46 \tC55 \tC56 \tC66\n')

c_vecs = []

for i in range(data_size):
    Cvec_Brown_random = np.around( np.random.normal(Cvec_Brown,Cvec_Brown_stdv), 1 )
    if write:
        with open('output.txt', 'a') as f:
            for value in Cvec_Brown_random:
                f.write(f'{value:.1f} \t')
            f.write('\n')
    c_vecs.append(Cvec_Brown_random)

In [ ]:
workers = os.cpu_count()
betas = np.empty((len(symmetry_classes), data_size))

distance_new = partial(distance, use_parallel_processing=False)

for i, sigma in enumerate(symmetry_classes):   
    print(f'running minimizer for symmetry class: {sigma}')
    with concurrent.futures.ProcessPoolExecutor(max_workers=workers) as executor:
        results = list(executor.map(distance_new, c_vecs, [sigma]*data_size))
    for j in range(data_size):
        betas[i,j] = results[j][0]
        
np.save('betas_brown', betas)

In [ ]:
# histograms

mean = np.zeros(len(symmetry_classes))
stdv = np.zeros(len(symmetry_classes))

fig, axs = plt.subplots(7,1,figsize=(10,30))

for j, sigma in enumerate(symmetry_classes):
    
    mean[j] = np.mean(betas[j,:])
    stdv[j] = np.std(betas[j,:])

stdv_max = np.max(stdv)

for j, sigma in enumerate(symmetry_classes):
    
    x1 = mean[j] - 3*stdv_max
    x2 = mean[j] + 3*stdv_max
    
    ax = axs[j]
    if j==0:
        ax.hist(betas[j,:], bins=50, label=f"An0")
        ax.legend(fontsize=30)
    else:
        ax.hist(betas[j,:], bins=50)
    ax.axvline(beta0[j],color='k')
    ax.axvline(mean[j],color='r')
    ax.axvline(mean[j]-2*stdv[j],color='r',linestyle='--')
    ax.axvline(mean[j]+2*stdv[j],color='r',linestyle='--')
    ax.set_xlim([x1, x2])
    ax.set_title(r'$\beta_{{{}}}$ ($\mu$ = {:.1f}, 2$\sigma$ = {:.1f})'.format(sigma, mean[j], 2*stdv[j]), 
                 fontsize = 30)
    ax.tick_params(axis='both', which='major', labelsize=30)

plt.subplots_adjust(hspace=0.5)
if write: plt.savefig(f"Brown_An0_{data_size}.png", bbox_inches='tight')
plt.show()

In [ ]:
# differential evolution computational time and number of outliers trade off analysis

pop_sizes = [15, 45, 100, 135, 200]
execution_times = np.array([689, 2339, 4879, 6735, 10366])/60
outlier_percent_MONO = [4.1, 0.0, 0.0, 0.0, 0.0]
outlier_percent_ORTH = [30.7, 9.2, 1.3, 0.7, 0.1]
outlier_percent_TET  = [0.0, 0.0, 0.0, 0.0, 0.0]
outlier_percent_CUBE = [0.0, 0.0, 0.0, 0.0, 0.0]
outlier_percent_TRIG = [9.7, 0.4, 0.1, 0.1, 0.1]
outlier_percent_XISO = [0.0, 0.0, 0.0, 0.0, 0.0]
outlier_percent_ISO  = [0.0, 0.0, 0.0, 0.0, 0.0]

plt.figure()
plt.scatter(pop_sizes, execution_times)
plt.xlabel('pop size')
plt.ylabel('execution time (minutes)')
plt.show()

plt.figure()
plt.scatter(pop_sizes, outlier_percent_MONO, label='MONO')
plt.scatter(pop_sizes, outlier_percent_ORTH, label='ORTH')
plt.scatter(pop_sizes, outlier_percent_TET, label='TET')
plt.scatter(pop_sizes, outlier_percent_CUBE, label='CUBE')
plt.scatter(pop_sizes, outlier_percent_TRIG, label='TRIG')
plt.scatter(pop_sizes, outlier_percent_XISO, label='XISO')
plt.scatter(pop_sizes, outlier_percent_ISO, label='ISO')
plt.xlabel('pop size')
plt.ylabel('outlier percentage (%)')
plt.legend()
plt.show()